Matthew Twete


Basic feedforward neural networks trained to classify the fashionMNIST dataset with various activations and hyperparameters. Models were also trained on polluted and shifted data.

In [ ]:
#Import libraries
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

#Download training data from open datasets.
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

#Download test data from open datasets.
test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [ ]:
batch_size = 30

#Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

In [ ]:
#Get cpu or gpu device for training.
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

#Define model for a feedforward MLP
class NeuralNetwork(nn.Module):
    #Updated constructor with arguments for the number layers and an optional
    #argument if you want the activation function to be sigmoid
    def __init__(self,numLayers,sig = False):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        if (numLayers == 2):
          if (sig == False):
            self.linear_relu_stack = nn.Sequential(
                nn.Linear(28*28, 1024),
                nn.ReLU(),
                nn.Linear(1024, 1024),
                nn.ReLU(),
                nn.Linear(1024, 10)
            )
          else:
            self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, 1024),
            nn.Sigmoid(),
            nn.Linear(1024, 10)
        )

        else:
          self.linear_relu_stack = nn.Sequential(
              nn.Linear(28*28, 1024),
              nn.ReLU(),
              nn.Linear(1024, 10)
          )
    #Define function to feed data forward
    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#Create the networks 
oneLayerNN = NeuralNetwork(1).to(device)
print(oneLayerNN)
TwoLayerNN = NeuralNetwork(2).to(device)
print(TwoLayerNN)

In [ ]:
#Define the loss function
loss_fn = nn.CrossEntropyLoss()

In [ ]:
#Define training function
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()


In [ ]:
#Define testing function
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [ ]:
#Train and test the two networks
epochs = 2
optimizer1 = torch.optim.SGD(oneLayerNN.parameters(), lr=1e-3)
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, oneLayerNN, loss_fn, optimizer1)
    test(test_dataloader, oneLayerNN, loss_fn)
optimizer2 = torch.optim.SGD(TwoLayerNN.parameters(), lr=1e-3)    
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, TwoLayerNN, loss_fn, optimizer2)
    test(test_dataloader, TwoLayerNN, loss_fn)
print("Done!")

In [ ]:
#Loop over all the batch sizes and learning rates for 2 epochs each 
#and test and train a 2 layer ReLU and 2 layer sigmoid model with those parameters 
batch_sizes = [1,10,100]
learn_rate = [1,0.1,0.01,0.001]
epochs = 2
#Loop over batch sizes
for size in batch_sizes:
  #Create data loaders
  train_dataloader = DataLoader(training_data, batch_size=size)
  test_dataloader = DataLoader(test_data, batch_size=size)
  #Loop over learning rates
  for rate in learn_rate:
    #Print the batch size and learning rate for the iteration
    print("Batch size: ", size, " Learning rate: ", rate)
    #Set up, train and test the ReLU and sigmoid networks
    ReLU_NN = NeuralNetwork(2).to(device)
    Sig_NN = NeuralNetwork(2,True).to(device)
    optimizer1 = torch.optim.SGD(ReLU_NN.parameters(), lr=rate)
    print("ReLU NN")
    for t in range(epochs):
      print(f"Epoch {t+1}\n-------------------------------")
      train(train_dataloader, ReLU_NN, loss_fn, optimizer1)
      test(test_dataloader, ReLU_NN, loss_fn)
    optimizer2 = torch.optim.SGD(Sig_NN.parameters(), lr=rate)
    print("Sigmoid NN")    
    for t in range(epochs):
      print(f"Epoch {t+1}\n-------------------------------")
      train(train_dataloader, Sig_NN, loss_fn, optimizer2)
      test(test_dataloader, Sig_NN, loss_fn)
    print("Done!")




In [ ]:
#Load a copy of the training data to pollute
polluted_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)


In [ ]:
#Import numpy
import numpy as np

#Create an array to keep track of how many polluted images of each
#class have been added to the other 9 classes. Each row represents a
#class, with each column representing the remaining number of instances 
#of that row's class that need to be added to the columns class
#Ex: row 0, column 1 represents how many instances of the 0th class need to be
#converted to the 1st class
pollution_counts = np.full((10,10),6)
#Since you can't pollute a class with itself, convert the diagonals to 0
np.fill_diagonal(pollution_counts,0)

In [ ]:
num_data = len(polluted_data.targets)
#Loop over all the data
for i in range(num_data):
  #Get the label for the i'th image 
  label = polluted_data.targets[i]
  #Check if all 9 other classes have been fully polluted by this class 
  #this will happen when the row in pollution_counts corrosponding to the class label
  #is all 0's, so if it is not all 0s, the label needs to be changed to pollute another 
  #class 
  if (np.count_nonzero(pollution_counts[label]) != 0):
    #Find one of the classes that needs to be polluted by the current image's class
    new_label = np.nonzero(pollution_counts[label])[0][0]
    #Decrement the number of remaining instances of that class that need to be polluted 
    #by the current image's class
    pollution_counts[label][new_label] -= 1
    #Change the label of the current image's class to the one that it needs to pollute
    polluted_data.targets[i] = new_label


In [ ]:
#Turns out the best network from before was the ReLU model with batch size of 1 and a learning rate of 0.01, 
#so create a new network with those parameters to train with polluted data
learn_rate = 0.01
batch_size = 1
polluted_model = NeuralNetwork(2).to(device)
optimizer1 = torch.optim.SGD(polluted_model.parameters(), lr=learn_rate)
epochs = 2
# Create data loader for the polluted training data & normal test data
train_dataloader = DataLoader(polluted_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)
#Train and test the network
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, polluted_model, loss_fn, optimizer1)
    test(test_dataloader, polluted_model, loss_fn)

In [ ]:
#I will continue to use the ReLU model with batch size of 1 and a learning rate of 0.01
#Due to the way I structured my code, I didn't save each model, so I will retrain one
#with the same parameters to use for part 5

#Set up and train the model
train_dataloader = DataLoader(training_data, batch_size=batch_size)
ReLU_NN = NeuralNetwork(2).to(device)
optimizer1 = torch.optim.SGD(ReLU_NN.parameters(), lr=learn_rate)
for t in range(epochs):
  train(train_dataloader, ReLU_NN, loss_fn, optimizer1)
  test(test_dataloader, ReLU_NN, loss_fn)

In [ ]:
#Load a copy of the test data to shift
shifted_test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

#Do a right circular shift on the data
shifted_test_data.data = torch.roll(shifted_test_data.data, shifts=(0,0,2), dims=(0,1,2))
#Create data loader
shifted_test_dataloader = DataLoader(shifted_test_data, batch_size=batch_size)
#Test the network on the shifted data
test(shifted_test_dataloader, ReLU_NN, loss_fn)
#Do a down circular shift and put the data in a dataloader
shifted_test_data.data = torch.roll(shifted_test_data.data, shifts=(0,2,0), dims=(0,1,2))
shifted_test_dataloader = DataLoader(shifted_test_data, batch_size=batch_size)
#Test again on the network
test(shifted_test_dataloader, ReLU_NN, loss_fn)